In [ ]:
import os

os.chdir("/home/cuitiancheng/ProG/")
%env CUDA_VISIBLE_DEVICES=0
from prompt_graph.tasker import NodeTask, GraphTask
from prompt_graph.utils import seed_everything
from torchsummary import summary
from prompt_graph.utils import print_model_parameters
from prompt_graph.utils import get_args
from prompt_graph.data import load4node, load4graph, split_induced_graphs
import pickle
import random
import numpy as np
import os
import pandas as pd

You can unzip the Experiment.zip to get the induced_graph file or run to get the induced_graph by yourself

In [ ]:
def load_induced_graph(dataset_name, data, device):

    folder_path = "./Experiment/induced_graph/" + dataset_name
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    file_path = folder_path + "/induced_graph_min100_max300.pkl"
    if os.path.exists(file_path):
        with open(file_path, "rb") as f:
            print("loading induced graph...")
            graphs_list = pickle.load(f)
            print("Done!!!")
    else:
        print("Begin split_induced_graphs.")
        split_induced_graphs(
            data, folder_path, device, smallest_size=100, largest_size=300
        )
        with open(file_path, "rb") as f:
            graphs_list = pickle.load(f)
    graphs_list = [graph.to(device) for graph in graphs_list]
    return graphs_list

set the parameters.

In [ ]:
class get_args:
    downstream_task = "NodeTask"  # 根据你的需求设置
    dataset_name = "Cora"
    device = 0
    gnn_type = "GCN"
    prompt_type = "GPF-plus"
    hid_dim = 128
    batch_size = 128
    epochs = 1000
    shot_num = 1
    pre_train_model_path = (
        "./Experiment/pre_trained_model/Cora/Edgepred_Gprompt.GCN.128hidden_dim.pth"
    )
    lr = 0.02
    decay = 2e-6
    num_layer = 2
    seed = 42

In [ ]:
args = get_args()
seed_everything(args.seed)

In [ ]:
if args.downstream_task == "NodeTask":
    data, input_dim, output_dim = load4node(args.dataset_name)
    print(output_dim)
    data = data.to(args.device)
    if args.prompt_type in ["Gprompt", "All-in-one", "GPF", "GPF-plus"]:
        graphs_list = load_induced_graph(args.dataset_name, data, args.device)
    else:
        graphs_list = None

In [ ]:


if args.downstream_task == "GraphTask":
    input_dim, output_dim, dataset = load4graph(args.dataset_name)

if args.downstream_task == "NodeTask":
    tasker = NodeTask(
        pre_train_model_path=args.pre_train_model_path,
        dataset_name=args.dataset_name,
        num_layer=args.num_layer,
        gnn_type=args.gnn_type,
        hid_dim=args.hid_dim,
        prompt_type=args.prompt_type,
        epochs=args.epochs,
        shot_num=args.shot_num,
        device=args.device,
        lr=args.lr,
        wd=args.decay,
        batch_size=args.batch_size,
        data=data,
        input_dim=input_dim,
        output_dim=output_dim,
        graphs_list=graphs_list,
    )

if args.downstream_task == "GraphTask":
    tasker = GraphTask(
        pre_train_model_path=args.pre_train_model_path,
        dataset_name=args.dataset_name,
        num_layer=args.num_layer,
        gnn_type=args.gnn_type,
        hid_dim=args.hid_dim,
        prompt_type=args.prompt_type,
        epochs=args.epochs,
        shot_num=args.shot_num,
        device=args.device,
        lr=args.lr,
        wd=args.decay,
        batch_size=args.batch_size,
        dataset=dataset,
        input_dim=input_dim,
        output_dim=output_dim,
    )
pre_train_type = tasker.pre_train_type


_, test_acc, std_test_acc, f1, std_f1, roc, std_roc, _, _ = tasker.run()

print("Final Accuracy {:.4f}±{:.4f}(std)".format(test_acc, std_test_acc))
print("Final F1 {:.4f}±{:.4f}(std)".format(f1, std_f1))
print("Final AUROC {:.4f}±{:.4f}(std)".format(roc, std_roc))